# Modelos personalizados

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, losses

## 0. Dataset y red neuronal

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test,y_test) = fashion_mnist.load_data()

In [16]:
model = Sequential()
model.add(layers.Flatten(input_shape=(28,28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))

## 1. Funcion de costo

In [4]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
def my_loss(y_true, y_pred):
  return loss(y_true, y_pred)

In [14]:
# SparseCategoricalCrossentropy
def my_loss(y_true, y_pred):
  loss = -tf.math.log(y_pred)
  loss = tf.gather_nd(loss, tf.cast(y_true,tf.int32), batch_dims=1)
  loss = tf.reduce_mean(loss)
  return loss

In [ ]:
def my_loss2(k):
  def my_loss(y_true, y_pred):
    loss = -tf.math.log(y_pred*k)
    loss = tf.gather_nd(loss, tf.cast(y_true,tf.int32), batch_dims=1)
    loss = tf.reduce_mean(loss)
    return loss
  return my_loss

In [19]:
class SCE(losses.Loss):
  def __init__(self, k=1, k2=0.99, **kwargs):
    super().__init__(**kwargs)
    self.k = k
    self.k2 = k2

  def call(self, y_true, y_pred):
    loss = -tf.math.log(y_pred*self.k)
    loss = tf.gather_nd(loss, tf.cast(y_true,tf.int32), batch_dims=1)
    loss = tf.reduce_mean(loss)*self.k2
    return loss

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, 'k':self.k, 'k2':self.k2}

In [17]:
model.compile(optimizer='adam',loss=my_loss,metrics=['acc'])
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
469/469 [==============================] - 9s 15ms/step - loss: 0.4777 - acc: 0.8289
Epoch 2/10
469/469 [==============================] - 7s 16ms/step - loss: 0.3536 - acc: 0.8701
Epoch 3/10
469/469 [==============================] - 9s 19ms/step - loss: 0.3140 - acc: 0.8827
Epoch 4/10
469/469 [==============================] - 9s 20ms/step - loss: 0.2869 - acc: 0.8938
Epoch 5/10
469/469 [==============================] - 8s 16ms/step - loss: 0.2704 - acc: 0.9000
Epoch 6/10
469/469 [==============================] - 7s 15ms/step - loss: 0.2520 - acc: 0.9050
Epoch 7/10
469/469 [==============================] - 7s 16ms/step - loss: 0.2376 - acc: 0.9106
Epoch 8/10
469/469 [==============================] - 9s 19ms/step - loss: 0.2258 - acc: 0.9140
Epoch 9/10
469/469 [==============================] - 10s 22ms/step - loss: 0.2130 - acc: 0.9188
Epoch 10/10
469/469 [==============================] - 9s 20ms/step - loss: 0.2055 - acc: 0.9222


## 2. Funciones de activacion, inicializadores, regularizadores

In [20]:
# Funcion de activacion personalizada
def my_elu(x):
  pos = x
  neg = tf.exp(x) - 1.0
  return tf.where(x>0, pos, neg)

In [21]:
# Inicializadores
def my_glorot_init(shape, dtype=tf.float32):
  sigma = tf.sqrt(2.0/(shape[0] + shape[1]))
  return tf.random.normal(shape, stddev=sigma, dtype=dtype)

In [22]:
# Regularizador
def my_regularizer(weights):
  return tf.reduce_sum(tf.abs(0.05*weights))

In [23]:
my_layer = layers.Dense(64, activation=my_elu, kernel_initializer=my_glorot_init,
                        kernel_regularizer=my_regularizer)

In [24]:
model = Sequential()
model.add(layers.Flatten(input_shape=(28,28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(my_layer)
model.add(layers.Dense(10, activation='softmax'))

In [25]:
model.compile(optimizer='adam',loss=my_loss,metrics=['acc'])
model.fit(x_train, y_train, epochs=10, batch_size=128)

Epoch 1/10
469/469 [==============================] - 14s 18ms/step - loss: 5.8047 - acc: 0.8254
Epoch 2/10
469/469 [==============================] - 8s 18ms/step - loss: 0.8189 - acc: 0.8678
Epoch 3/10
469/469 [==============================] - 8s 17ms/step - loss: 0.6442 - acc: 0.8827
Epoch 4/10
469/469 [==============================] - 8s 18ms/step - loss: 0.5617 - acc: 0.8929
Epoch 5/10
469/469 [==============================] - 8s 17ms/step - loss: 0.5065 - acc: 0.9012
Epoch 6/10
469/469 [==============================] - 8s 17ms/step - loss: 0.4638 - acc: 0.9088
Epoch 7/10
469/469 [==============================] - 8s 17ms/step - loss: 0.4341 - acc: 0.9119
Epoch 8/10
469/469 [==============================] - 8s 16ms/step - loss: 0.4135 - acc: 0.9168
Epoch 9/10
469/469 [==============================] - 8s 17ms/step - loss: 0.3843 - acc: 0.9233
Epoch 10/10
469/469 [==============================] - 8s 17ms/step - loss: 0.3658 - acc: 0.9264


## 3. Metricas personalizada

In [ ]:
class MyMetric(tf.keras.metrics.Metric):
  def __init__(self, k=0.1, **kwargs):
    super().__init__(**kwargs)
    pass
  
  def update_state(self,y_true, y_pred, sample_weight=None):
    metric = self.metric(y_true, y_pred)
    self.total.assign_add(tf.reduce_mean(metric))
  
  def result(self):
    return self.total

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, 'k':self.k}

## 4. Capas personalizadas

In [26]:
class MyLayer(layers.Layer):
  def __init__(self, units, activation=None, **kwargs):
    self.units = units
    self.activation = tf.keras.activations.get(activation)
    super().__init__(**kwargs)

  def build(self, batch_input_shape):
    self.weights = self.add_weight(name='weights', shape=[batch_input_shape[-1], self.units],
                                   initializer='glorot_normal')
    self.bias = self.add_weight(name='bias', shape=[self.units], initializer='glorot_normal')
    super().build(batch_input_shape)
  
  def call(self, X):
    return self.activation(X @ self.weights + self.bias)

  def compute_output_shape(self, batch_input_shape):
    return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, 'units': self.units, 'activation': tf.keras.activations.serialize(self.activation)}

## 5. Modelos personalizados

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.input = layers.Dense(30, activation='elu', input_shape=(10,))
    self.dense1 = layers.Dense(32, activation='relu')
    self.dense1 = layers.Dense(5, activation='softmax')
    pass
  
  def call(self, inputs):
    x = self.input(inputs)
    x = self.dense(x)
    x = self.dense1(x)
    return x

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, }

## 6. Gradientes en Tensorflow

In [27]:
def func(w1,w2):
  return 5 + w1**3 + w2**2 + w1*w2 

In [40]:
w1, w2 = 3, 2
eps = 1e-8

In [41]:
(func(w1+eps,w2)-func(w1,w2))/eps

29.00000026784255

In [42]:
(func(w1,w2+eps)-func(w1,w2))/eps

7.000000579182597

Autodiff

In [43]:
w1, w2 = tf.Variable(3.), tf.Variable(2.)
with tf.GradientTape() as tape:
  #tape.watch(w1)
  z = func(w1,w2)

gradient = tape.gradient(z,[w1, w2])

In [44]:
print(gradient)

[<tf.Tensor: shape=(), dtype=float32, numpy=29.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>]


In [54]:
x = tf.Variable(100.0)
with tf.GradientTape() as tape:
  x_r = tf.math.log(tf.exp(x)+1)

tape.gradient(x_r,[x])

[<tf.Tensor: shape=(), dtype=float32, numpy=nan>]

In [65]:
@tf.custom_gradient
def Mysoftplus(x):
  x_exp = tf.exp(x)
  def gradient(grad):
    return grad/(1+1/x_exp)
  return tf.math.log(x_exp+1), gradient

In [66]:
x = tf.Variable(0.0)
with tf.GradientTape() as tape:
  x_r = Mysoftplus(x)

tape.gradient(x_r,[x])

[<tf.Tensor: shape=(), dtype=float32, numpy=0.5>]

## 7. Entrenamiento personalizado

In [72]:
n_epochs = 10
batch_size = 128
n_steps = len(x_train) // batch_size
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.Accuracy()
mean_loss = tf.keras.metrics.Mean()

In [70]:
# Barra de status
def status_bar(iteration, total, loss, metrics=None):
  metrics = ' - '.join(['{}:{:.4f}'.format(v.name, v.result()) for v in [loss]+(metrics or [])])
  end = "" if iteration<total else "\n"
  print("\r{}/{} - ".format(iteration,total) + metrics, end=end)

In [74]:
import numpy as np
def get_batch(x,y, batch_size):
  idx = np.random.randint(len(x), size=batch_size)
  return x[idx], y[idx]

In [80]:
loss_list = []
for epoch in range(1, n_epochs + 1):
  print('Epoch {}/{}'.format(epoch, n_epochs))
  for step in range(n_steps):
    X_batch, Y_batch = get_batch(x_train,y_train, batch_size)
    with tf.GradientTape() as tape:
      y_pred = model(X_batch, training=True)
      loss = tf.reduce_mean(loss_fn(Y_batch, y_pred))
      loss_list.append(loss) # Se puede modificar
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    mean_loss(loss)
    odds = y_pred
    y_pred = tf.math.argmax(odds, axis=1)
    metric(Y_batch, y_pred)
    status_bar(step+1, n_steps, mean_loss, [metric])
  
  metric.reset_states()

Epoch 1/10
468/468 - mean:0.1957 - accuracy:0.9332
Epoch 2/10
468/468 - mean:0.1902 - accuracy:0.9363
Epoch 3/10
468/468 - mean:0.1858 - accuracy:0.9394
Epoch 4/10
468/468 - mean:0.1815 - accuracy:0.9417
Epoch 5/10
468/468 - mean:0.1775 - accuracy:0.9438
Epoch 6/10
443/468 - mean:0.1745 - accuracy:0.9439

KeyboardInterrupt: ignored

## 8. Funciones en tensorflow

In [87]:
def func(x):
  return x**2 + 5*x + 2

In [83]:
tf_func = tf.function(func)
tf_func 

In [84]:
tf_func(1.0)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [85]:
@tf.function
def func(x):
  return x**2 + 5*x + 2

## 9. Graficos en tensorflow

Autograph

In [88]:
def func(x):
  return x**2 + 5*x + 2

In [92]:
graph = tf.autograph.to_graph(func)

In [90]:
trace = tf.autograph.trace(func)

<function func at 0x7f1ecae42d40>
